In [1]:
import os
import datetime
import time
import google.generativeai as genai
from google.generativeai import caching
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
import json

# Load environment variables
load_dotenv()

# Retrieve API keys from environment variables
google_api_key = os.getenv("GOOGLE_API_KEY")
firecrawl_api_key = os.getenv("FIRECRAWL_API_KEY")

# Configure the Google Generative AI module with the API key
genai.configure(api_key=google_api_key)

# Initialize the FirecrawlApp with your API key
app = FirecrawlApp(api_key=firecrawl_api_key)


/Users/ericciarla/projects/python_projects/agents_testing/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Crawl a website
crawl_url = 'https://dify.ai/'
params = {
   
    'crawlOptions': {
        'limit': 100
    }
}
crawl_result = app.crawl_url(crawl_url, params=params)

if crawl_result is not None:
    # Convert crawl results to JSON format, excluding 'content' field from each entry
    cleaned_crawl_result = [{k: v for k, v in entry.items() if k != 'content'} for entry in crawl_result]

    # Save the modified results as a text file containing JSON data
    with open('crawl_result.txt', 'w') as file:
        file.write(json.dumps(cleaned_crawl_result, indent=4))
else:
    print("No data returned from crawl.")


No data returned from crawl.


In [10]:
# Upload the video using the Files API
text_file = genai.upload_file(path="crawl_result.txt")

# Wait for the file to finish processing
while text_file.state.name == "PROCESSING":
    print('Waiting for file to be processed.')
    time.sleep(2)
    text_file = genai.get_file(text_file.name)



In [11]:
# Create a cache with a 5 minute TTL
cache = caching.CachedContent.create(
    model="models/gemini-1.5-pro-002",
    display_name="website crawl testing again", # used to identify the cache
    system_instruction="You are an expert at this website, and your job is to answer user's query based on the website you have access to.",
    contents=[text_file],
    ttl=datetime.timedelta(minutes=15),
)
# Construct a GenerativeModel which uses the created cache.
model = genai.GenerativeModel.from_cached_content(cached_content=cache)



In [13]:
# Query the model
response = model.generate_content(["What powers website scraping with Dify?"])
response_dict = response.to_dict()
response_text = response_dict['candidates'][0]['content']['parts'][0]['text']
print(response_text)


Dify.AI utilizes the **Firecrawl** service for website scraping. This service can crawl and convert any website into clean markdown or structured data that's ready for use in building RAG applications. 

Here's how Firecrawl helps:

* **Crawling and Conversion:** Firecrawl crawls the website and converts the content into a format that is easily understood by LLMs, such as markdown or structured data.
* **Clean Output:**  Firecrawl ensures the data is clean and free of errors, making it easier to use in Dify's RAG engine.
* **Parallel Crawling:**  Firecrawl efficiently crawls web pages in parallel, delivering results quickly.

You can find Firecrawl on their website: [https://www.firecrawl.dev/](https://www.firecrawl.dev/)

Firecrawl offers both a cloud service and an open-source software (OSS) edition. 

